# Importação dos dados

In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
path = Path(os.getcwd())
path_parent = path.parent.absolute()
os.chdir(path_parent)

In [2]:
df = pd.read_csv(r'data\sales_data.csv', index_col = 0)

# Initial Profilling

In [7]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Profiling Sales Data")

In [10]:
profile.to_file('sales_data_report.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 40.70it/s]


# Dependencia da Hotmart com maiores produtores

1- Trazer essas informações para um Dashboard interativo

2- Fazer relação com distribuições de caudas longas 80/20

3- Mostrar relação dos que possuem um grande volume de vendas com a importância deles no lucro da HotMart

4- Mostrar relação dos que possuem um valor alto de vendas com a importância deles no lucro da Hotmart

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['purchase_transformed'] = scaler.fit_transform(df[['purchase_value']])
sum_of_values = df.groupby(['producer_id']).agg(valor_total = ('purchase_value', 'sum'),
                                                valor_transformed = ('purchase_transformed', 'sum'),
                                                count = ('purchase_transformed', 'count')).reset_index()
labels_cut = ['baixissimo_volume', 'baixo_volume', 'medio_volume', 'alto_volume', 'altissimo_volume']
sum_of_values['total_qcut'] = pd.qcut(sum_of_values['valor_total'], q=5, labels=labels_cut)
sum_of_values['transformed_qcut'] = pd.qcut(sum_of_values['valor_transformed'], q=5, labels=labels_cut)

In [81]:
np.percentile(sum_of_values['count'], 95)


730.1999999999971

In [83]:
hotmart_maiores_produtores = df.merge(sum_of_values, how='inner', left_on='producer_id', right_on='producer_id')

In [84]:
import plotly.express as px
top_20_value = sum_of_values[sum_of_values['valor_transformed'] > np.percentile(sum_of_values['valor_transformed'], 80)]['producer_id']
hotmart_maiores_produtores[hotmart_maiores_produtores['producer_id'].isin(top_20_value)]['purchase_transformed'].sum()/hotmart_maiores_produtores['purchase_transformed'].sum()

top_20_value = sum_of_values[sum_of_values['count'] > np.percentile(sum_of_values['count'], 80)]['producer_id']
hotmart_maiores_produtores[hotmart_maiores_produtores['producer_id'].isin(top_20_value)]['purchase_transformed'].sum()/hotmart_maiores_produtores['purchase_transformed'].sum()



0.9437371612326654

# Quais características impactam no sucesso de um produto (Quem são esses maiores produtores)

Adicionais a serem trazidos na análise

1- Verificar sazonalidade de tipos de produto

1.1 - Dias da semana com maior volume de compras

1.2 - Horários com maior volume de compras

2- Fazer a feature importance por tipo de produto

3- Fazer modelo que prediz quantas compras vão ser realizadas a um dado produto

4- Fazer página do Dash com um conjunto de opções e a partir disso medir/metrificar número de compras de produto ou total acumulado por produto

5- Fazer primeiro a previsão inicial para depois seguir para Feature Engineering

6- Fazer análise mais aprofundada de purchase_device, purchase_origin, is_origin_page_social_network

In [18]:
df = df[['product_id', 'affiliate_id', 'producer_id', 'buyer_id', 'purchase_date', 
    'product_creation_date', 'product_category', 'product_niche', 'purchase_value', 
    'affiliate_commission_percentual', 'purchase_device', 'purchase_origin', 
    'is_origin_page_social_network', 'purchase_transformed']]

## Impacto do Affiliate_id no total de vendas

In [36]:
df[df['purchase_origin'].str.contains('Origin')]['purchase_origin']

Series([], Name: purchase_origin, dtype: object)

In [25]:
prod_affiliate = df.groupby(['product_id', 'affiliate_id']).agg(total_de_vendas = ('product_id', 'count'),
                                               valor_de_vendas = ('purchase_transformed', 'sum')
                                               ).reset_index()
affiliate = df.groupby(['affiliate_id']).agg(total_de_vendas = ('product_id', 'count'),
                                               valor_de_vendas = ('purchase_transformed', 'sum')
                                               ).reset_index()

In [27]:
affiliate[]

,affiliate_id,total_de_vendas,valor_de_vendas
0,3,99,0.039555
1,59,4,0.015018
2,60,1599,2.854548
3,62,151,0.780802
4,80,498,2.425862
...,...,...,...
22942,7678701,1,0.000899
22943,7678978,1,0.001019
22944,7683014,1,0.003736
22945,7689657,1,0.014960


In [92]:
prod_buy = df.groupby(['product_id', 'buyer_id']).agg(count = ('purchase_id', 'count'))
prod_buy[prod_buy['count'] > 2]
df[(df['product_id'] == 3336) & (df['buyer_id'] == 2699)]

,purchase_id,product_id,affiliate_id,producer_id,buyer_id,purchase_date,product_creation_date,product_category,product_niche,purchase_value,affiliate_commission_percentual,purchase_device,purchase_origin,is_origin_page_social_network,Venda,purchase_transformed
222396,11288222,3336,50788,50788,2699,2016-01-28 19:49:52,2010-10-16 10:21:09,Phisical book,Presentation skills,-0.432554,0.0,Smart TV,Origin ef2b,"0,0",1,0.000869
465362,11789529,3336,50788,50788,2699,2016-02-28 17:50:02,2010-10-16 10:21:09,Phisical book,Presentation skills,-0.432554,0.0,Smart TV,Origin ef2b,"0,0",1,0.000869
720754,12306047,3336,50788,50788,2699,2016-03-28 17:30:21,2010-10-16 10:21:09,Phisical book,Presentation skills,-0.432554,0.0,Smart TV,Origin ef2b,"0,0",1,0.000869
991408,12846906,3336,50788,50788,2699,2016-04-28 17:25:47,2010-10-16 10:21:09,Phisical book,Presentation skills,-0.432554,0.0,Smart TV,Origin ef2b,"0,0",1,0.000869
1281678,13393547,3336,50788,50788,2699,2016-05-28 17:11:59,2010-10-16 10:21:09,Phisical book,Presentation skills,-0.432554,0.0,Smart TV,Origin ef2b,"0,0",1,0.000869
1581627,13975925,3336,50788,50788,2699,2016-06-28 17:41:21,2010-10-16 10:21:09,Phisical book,Presentation skills,-0.432554,0.0,Smart TV,Origin ef2b,"0,0",1,0.000869


In [85]:
df.groupby(by=['product_id', 'affiliate_id', 'producer_id', 'buyer_id', 'purchase_date', ])

,purchase_id,product_id,affiliate_id,producer_id,buyer_id,purchase_date,product_creation_date,product_category,product_niche,purchase_value,affiliate_commission_percentual,purchase_device,purchase_origin,is_origin_page_social_network,Venda,purchase_transformed
0,1663958,6640,209372,116238,1200397,2016-06-26 12:00:00,2011-03-19 15:47:36,Video,Presentation skills,-0.265302,NaN,Smart TV,Origin ef2b,"0,0",1,0.002206
1,1677087,2350,141418,2821,1083764,2016-06-26 12:00:00,2010-07-05 01:50:15,Podcast,Child psychology,-0.177077,NaN,Smart TV,Origin ef2b,"0,0",1,0.002911
2,2017360,35669,618642,618642,1436106,2016-06-26 12:00:00,2012-06-13 02:59:37,Podcast,Presentation skills,-0.468989,NaN,Smart TV,Origin ef2b,"0,0",1,0.000578
3,2017379,57998,1164511,70388,1436118,2016-06-26 12:00:00,2013-05-07 08:51:31,Podcast,Anxiety management,-0.401168,NaN,Smart TV,Origin ef2b,"0,0",1,0.001120
4,2017382,58329,1261488,221253,1386357,2016-06-26 12:00:00,2013-05-12 08:12:06,Podcast,Teaching English,-0.452489,NaN,Smart TV,Origin ef2b,"0,0",1,0.000709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599823,14011995,238362,7586641,7586641,5736172,2016-06-30 23:59:57,2016-06-16 12:10:46,Phisical book,Personal finance,-0.345361,0.0,eReaders,Origin 3022,"0,0",1,0.001566
1599824,14012431,61279,589022,589022,946067,2016-06-30 21:40:11,2013-06-15 16:41:06,Phisical book,Personal finance,-0.471786,0.0,Smart TV,Origin ef2b,"0,0",1,0.000555
1599825,14343996,215242,1186145,1186145,6473172,2016-05-13 16:45:42,2016-03-26 17:59:47,Phisical book,Negotiation,-0.359158,0.0,Smart TV,Origin ef2b,"0,0",1,0.001455
1599826,14344113,215242,1186145,1186145,6473172,2016-06-22 14:39:05,2016-03-26 17:59:47,Phisical book,Negotiation,-0.359158,0.0,Smart TV,Origin ef2b,"0,0",1,0.001455


In [55]:
hotmart_maiores_produtores[~hotmart_maiores_produtores['transformed_qcut'].isin(remove)]

,purchase_id,product_id,affiliate_id,producer_id,buyer_id,purchase_date,product_creation_date,product_category,product_niche,purchase_value,affiliate_commission_percentual,purchase_device,purchase_origin,is_origin_page_social_network,Venda,purchase_transformed,valor_total,valor_transformed,total_qcut,transformed_qcut
2381,2017360,35669,618642,618642,1436106,2016-06-26 12:00:00,2012-06-13 02:59:37,Podcast,Presentation skills,-0.468989,NaN,Smart TV,Origin ef2b,"0,0",1,0.000578,-0.953915,0.001028,medio_volume,baixissimo_volume
2382,2017656,58535,618642,618642,1211642,2016-06-26 12:00:00,2013-05-15 03:02:52,Podcast,Presentation skills,-0.484926,NaN,Smart TV,Origin ef2b,"0,0",1,0.000450,-0.953915,0.001028,medio_volume,baixissimo_volume
2985,2017382,58329,1261488,221253,1386357,2016-06-26 12:00:00,2013-05-12 08:12:06,Podcast,Teaching English,-0.452489,NaN,Smart TV,Origin ef2b,"0,0",1,0.000709,-0.452489,0.000709,alto_volume,baixissimo_volume
3113,2017442,36468,186897,186897,1252324,2016-06-26 12:00:00,2012-06-25 19:29:42,Podcast,Presentation skills,-0.472931,NaN,Smart TV,Origin ef2b,"0,0",1,0.000546,-0.472931,0.000546,alto_volume,baixissimo_volume
3116,2017569,49928,86355,86355,4604,2016-06-26 12:00:00,2013-01-05 12:18:22,Podcast,Media training,-0.528100,NaN,Smart TV,Origin ef2b,"0,0",1,0.000105,-0.528100,0.000105,medio_volume,baixissimo_volume
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599809,14003791,197942,2222578,2222578,1438702,2016-06-30 12:02:56,2016-01-16 15:29:11,Podcast,Personal finance,-0.503754,0.0,eReaders,Origin a599,"0,0",1,0.000300,-0.503754,0.000300,medio_volume,baixissimo_volume
1599812,14008316,239138,7472399,7472399,7699280,2016-06-30 18:00:45,2016-06-19 19:20:28,Phisical book,Presentation skills,-0.490557,0.0,eReaders,Origin 5109,"0,0",1,0.000405,-0.490557,0.000405,alto_volume,baixissimo_volume
1599819,14009300,191091,5616311,5616311,7700067,2016-06-30 19:33:48,2015-12-11 06:16:13,Podcast,Presentation skills,-0.533750,0.0,Smart TV,Origin ef2b,"0,0",1,0.000060,-0.533750,0.000060,medio_volume,baixissimo_volume
1599820,14009400,238703,7598719,7598719,5307000,2016-06-30 19:44:59,2016-06-18 14:30:54,Phisical book,Economics,-0.372486,0.0,eReaders,Origin adf0,"0,0",1,0.001349,-0.372486,0.001349,alto_volume,baixissimo_volume
